# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1
Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4.

#### Create DB

In [8]:
query = "CREATE TABLE IF NOT EXISTS music_app_history"
query = query + "(session_id int, item_in_session int, artist_name text, song_title text,song_length decimal,\
PRIMARY KEY (session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)                 

#### Insert Data

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO music_app_history (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9],float(line[5])))

#### Query Data

In [11]:
query = """
SELECT artist_name, song_title, song_length FROM music_app_history \
WHERE session_id =338 AND item_in_session = 4"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


## Query 2
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Create DB

In [12]:
query = "CREATE TABLE IF NOT EXISTS music_app_history1"
query = query + "(user_id int,session_id int, item_in_session int, artist_name text, song_title text,\
PRIMARY KEY ((user_id,session_id), item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)

#### Insert Data

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO music_app_history1 (user_id,session_id, item_in_session, artist_name, song_title)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9]))

#### Query Data

In [14]:
query = """
SELECT artist_name, song_title, item_in_session FROM music_app_history1 \
WHERE user_id =10 AND session_id = 182"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.artist_name, row.song_title, row.item_in_session)

Down To The Bone Keep On Keepin' On 0
Three Drives Greece 2000 1
Sebastien Tellier Kilometer 2
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 3


## Query 3
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Create DB

In [21]:
query = "CREATE TABLE IF NOT EXISTS music_app_history2"
query = query + "(song_title text, first_name_user text, last_name_user text,\
PRIMARY KEY (song_title, first_name_user, last_name_user))"

try:
    session.execute(query)
except Exception as e:
    print(e)

#### Insert Data

In [22]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO music_app_history2 (song_title, first_name_user, last_name_user)"
        query = query + "VALUES (%s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4]))

#### Query Data

In [26]:
query = """
SELECT first_name_user, last_name_user FROM music_app_history2 \
WHERE song_title = 'All Hands Against His Own'"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.first_name_user, row.last_name_user)

Jacqueline Lynch
Sara Johnson
Tegan Levine


### Drop the tables before closing out the sessions

In [22]:
query = "DROP TABLE music_app_history"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE music_app_history1"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE music_app_history2"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()